In [ ]:
import ipyparallel as ipp

from ipyparallel import Cluster
cluster = await Cluster(engines="mpi", profile="myprofile").start_and_connect(n=4, activate=True)
cluster.ids

In [ ]:
%%px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve as ngs
import NgsAMG as amg

## HDiv AMG

Non-standard monolithic AMG method for Stokes, robust in domain shape and div-div penaly.

Similar to [ParELAG](https://github.com/LLNL/parelag), with simpler caorsening (*), fast smoothers, some tricks for parallel.


Should just work with H($\operatorname{div}$)-HDG/MCS, includes treatment for tangential trace.



(*) see also "Algebraic multigrid for k‐form Laplacians" (Luke Olson, Nathan Bell).

In [ ]:
%%px
from usrMtgStuffPar import GetValve, StokesHDGDiscretization, solveCondensed

mesh = GetValve(N=4, dim=2, maxh=0.1, closevalve=False)
uin = CF((1, 0)) if mesh.ngmesh.dim == 2 else CF((1,0,0))

wall = "wall" if mesh.ngmesh.dim == 3 else "default"
(V, a, f, u) = StokesHDGDiscretization(mesh, order=2, wall=wall, inlet="inlet", outlet="outlet", nu=1e-3, div_div_pen=1e6)

amg_cl = amg.stokes_hdiv_gg_2d if V.mesh.ngmesh.dim == 2 else amg.stokes_hdiv_gg_3d

c = amg_cl(a, ngs_amg_pres_vecs="P1", ngs_amg_sm_type="hiptmair", ngs_amg_sm_type_range="dyn_block_gs", ngs_amg_log_level="basic")

f.Assemble()
a.Assemble()

u.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
solveCondensed(a, c, u.vec, f.vec, tol=1e-8)

Draw(u.components[0], vectors=True)


The principle is to mimick geometric MG by constructing:
* a series of $L^2$-like spaces
$$Q_0 \xleftarrow[P^Q_0]{} Q_1 \xleftarrow[P^Q_0]{} Q_2, \ldots$$
* a series of $H(\operatorname{div})$-like spaces
$$V_0 \xleftarrow[P_0]{} V_1 \xleftarrow[P_0]{} V_2, \ldots$$
such that $\operatorname{div}(V_i) = Q_i$
* a series of $H^1/H(\operatorname{curl})$-like potential spaces $W_0 , W_1 , W_2, \ldots$
* discrete $\nabla^T/\operatorname{curl}$-operators $D_i: W_i \rightarrow V_i$ and discrete $\operatorname{div}$-operators $C_i: V_i \rightarrow Q_i$ such that on each level we have an exact sequence
  $$ W_i \xrightarrow[D_i]{} V_i \xrightarrow[C_i]{} Q_q \xrightarrow[Id]{} \{0\}$$
* Between $L^2$ and $H(\operatorname{div})$ space, prolongation and divergence commute
  $$P^Q_i C_{i+1} = C_{i} P_i$$
  

This is achieved by
* setting up the dual graph - elements become vertices, facets become edges, vertices/edges become "faces", or "loops"
* coarsening of vertices in this dual graph induces coarsening of loops
* construction of coarse basis functions with piecewise constant divergence via discrete harmonic extensions
* construction of coarse potential space/discrete $\nabla^T/\operatorname{curl}$-operator based on loops (correspond to vertices/edges in 2d,3d)



### Base functions

Going back to a simpler geometry for visualization

In [ ]:
%%px
from netgen.geom2d import unit_square
from usrMtgStuffPar import StokesHDGDiscretization

mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

(V, a, f, u) = StokesHDGDiscretization(mesh, order=2, wall="", inlet="", outlet=".*", nu=1e-3, div_div_pen=1e6)

c = amg.stokes_hdiv_gg_2d(a, ngs_amg_pres_vecs="P1", ngs_amg_sm_type="hiptmair", ngs_amg_max_coarse_size=1, ngs_amg_crs_alg="spw", ngs_amg_sm_type_range="dyn_block_gs", ngs_amg_log_level="basic")

a.Assemble()


In [ ]:
%%px
bf = GridFunction(V)

@interact(level=widgets.IntSlider(min=0, max=c.GetNLevels()-1, step=1, value=c.GetNLevels()-1, continuous_update=False), \
          drawDiv=True)
def getDrawBF(level, drawDiv):
    @interact(dof=widgets.IntSlider(min=0, max=c.GetNDof(level=level), step=1, value=0, continuous_update=False))
    def getDrawBFInner(dof):
        c.GetBF(vec=bf.vec, level=level, dof=dof, comp=0)
        if drawDiv:
            Draw(div(bf.components[0]), mesh)
        else:
            Draw(bf.components[0], vectors=True)


### Potential Space

In [ ]:
%%px
bf = GridFunction(V)
# Draw(bf.components[0], vectors=True)
# Draw(div(bf.components[0]), mesh)

@interact(level=widgets.IntSlider(min=0, max=c.GetNLevels()-2, step=1, value=c.GetNLevels()-2, continuous_update=False), \
          loop=widgets.IntSlider(min=0, max=30, step=1, value=0, continuous_update=False),\
          drawDiv=False)
def getDrawLoop(level, drawDiv):
    @interact(loop=widgets.IntSlider(min=0, max=c.GetNLoops(level), step=1, value=0, continuous_update=False))
    def getDrawLoopInner(loop):
        c.GetLoop(comp_vec=bf.vec, level=level, loop_num=loop)
        if drawDiv:
            Draw(div(bf.components[0]), mesh)
        else:
            Draw(bf.components[0], vectors=True)
    # Redraw()

